# ***Geospatial Indexing***

*This example shows how to create and use a GEO2D index in PyMongo. To create a spherical (earth-like) geospatial index use GEOSPHERE instead.*

In [1]:
# creating a geospatial index
from pymongo import MongoClient, GEO2D
db = MongoClient().geo_example
db.places.create_index([("loc", GEO2D)])

'loc_2d'

In [2]:
# Inserting places
# Locations in MongoDB are represented using either embedded documents or lists where the first two elements are coordinates. Here, we’ll insert a couple of example locations
result = db.places.insert_many(
    [{"loc": [2,5]}, {"loc": [30,5]}, {"loc": [1,2]},{"loc": [4,4]}]
)
result.inserted_ids

[ObjectId('64bcc95395fe54cb411ce9a7'),
 ObjectId('64bcc95395fe54cb411ce9a8'),
 ObjectId('64bcc95395fe54cb411ce9a9'),
 ObjectId('64bcc95395fe54cb411ce9aa')]

### *Querying*

In [3]:
# using the geospatial index we can find documents near another point
import pprint
for doc in db.places.find({"loc": {"$near": [3,6]}}).limit(3):
    pprint.pprint(doc)

{'_id': ObjectId('64bcc95395fe54cb411ce9a7'), 'loc': [2, 5]}
{'_id': ObjectId('64bcc95395fe54cb411ce9aa'), 'loc': [4, 4]}
{'_id': ObjectId('64bcc95395fe54cb411ce9a9'), 'loc': [1, 2]}


In [4]:
from bson.son import SON 
query = {"loc": SON([("$near", [3, 6]), ("$maxDistance", 100)])}
for doc in db.places.find(query).limit(3):
    pprint.pprint(doc)

{'_id': ObjectId('64bcc95395fe54cb411ce9a7'), 'loc': [2, 5]}
{'_id': ObjectId('64bcc95395fe54cb411ce9aa'), 'loc': [4, 4]}
{'_id': ObjectId('64bcc95395fe54cb411ce9a9'), 'loc': [1, 2]}


In [5]:
query = {"loc": {"$within": {"$box": [[2, 2], [5, 6]]}}}
for doc in db.places.find(query).sort("_id"):
    pprint.pprint(doc)

{'_id': ObjectId('64bcc95395fe54cb411ce9a7'), 'loc': [2, 5]}
{'_id': ObjectId('64bcc95395fe54cb411ce9aa'), 'loc': [4, 4]}


In [6]:
query = {"loc": {"$within": {"$center": [[0, 0], 6]}}}
for doc in db.places.find(query).sort("_id"):
    pprint.pprint(doc)

{'_id': ObjectId('64bcc95395fe54cb411ce9a7'), 'loc': [2, 5]}
{'_id': ObjectId('64bcc95395fe54cb411ce9a9'), 'loc': [1, 2]}
{'_id': ObjectId('64bcc95395fe54cb411ce9aa'), 'loc': [4, 4]}


In [7]:
from bson.son import SON
db.command(SON([('geoNear', 'places'), ('near', [1, 2])]))

{'results': [{'dis': 0.0,
   'obj': {'_id': ObjectId('64bcc95395fe54cb411ce9a9'), 'loc': [1, 2]}},
  {'dis': 3.1622776601683795,
   'obj': {'_id': ObjectId('64bcc95395fe54cb411ce9a7'), 'loc': [2, 5]}},
  {'dis': 3.605551275463989,
   'obj': {'_id': ObjectId('64bcc95395fe54cb411ce9aa'), 'loc': [4, 4]}},
  {'dis': 29.154759474226502,
   'obj': {'_id': ObjectId('64bcc95395fe54cb411ce9a8'), 'loc': [30, 5]}}],
 'stats': {'nscanned': 47,
  'objectsLoaded': 4,
  'avgDistance': 8.980647102464717,
  'maxDistance': 29.154759474226502,
  'time': 2428},
 'ok': 1.0}